In [1]:
!git clone https://github.com/satyaNekkantiCompVison/pytorch_visionmodels.git

fatal: destination path 'pytorch_visionmodels' already exists and is not an empty directory.


In [2]:
!pip install --upgrade albumentations

     |████████████████████████████████| 102 kB 5.3 MB/s 
     |████████████████████████████████| 47.6 MB 3.5 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


# Change Directory to main repo

In [3]:
cd pytorch_visionmodels/

/content/pytorch_visionmodels


## Get Helper functions

In [4]:
from utils.helper import seed_everything, get_device, model_summary
seed_everything(42)
device = get_device()

CUDA Available: True


## Loading Train and Test loader

In [5]:
from utils.data_augmentations import get_train_loader, get_test_loader
train_loader = get_train_loader(BATCH_SIZE=128)
test_loader = get_test_loader(BATCH_SIZE=128)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Resnet 18 Model summary with LN

In [6]:
from models.resnet import resnet18
from utils.helper import model_summary

model = resnet18("LN").to(device)

model_summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
         GroupNorm-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
         GroupNorm-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
         GroupNorm-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
         GroupNorm-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
        GroupNorm-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
        GroupNorm-14          [-1, 128,

## Training and Testing model


In [ ]:
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

from main import fit_model
epochs = 40
lr = 0.001

optimizer = optim.Adam(model.parameters(), lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min', patience=8, verbose=True)
criterion = nn.CrossEntropyLoss()

experiment_parameters = fit_model(model, device, epochs, optimizer,criterion, train_loader,test_loader,scheduler)

EPOCH: 1 (LR: 0.001)


Batch_id=54 Loss=261.20297 Accuracy=10.27%:  14%|█▍        | 55/391 [00:39<03:56,  1.42it/s]

## Plotting graphs



In [ ]:
from utils.plot_images import plot_axis
train_Acc,train_Loss, test_Acc, test_Loss = experiment_parameters

plot_axis(train_Loss, test_Loss, train_Acc,test_Acc)

## Plotting misclassified images

In [ ]:
from utils.plot_images import misclassified_images
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

images_misclassified = misclassified_images(model, test_loader, device)

## GradCAM output for misclassified images

In [ ]:
# from utils.gradcam import generate_gradcam, plot_gradcam
target_layers = ["layer1", "layer2", "layer3", "layer4"]
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
gradcam_output, probs, predicted_classes = generate_gradcam(images_misclassified[:20], model, target_layers, device)
plot_gradcam(gradcam_output, target_layers, labels, (3, 32, 32), predicted_classes, images_misclassified[:20])